In [1]:
%%time
%%bash

. ./cmd.sh
. ./path.sh
set -e

local/fph_data_prep.sh data/FSC

FSC data preparation succeeded.
utils/fix_data_dir.sh: filtered data/train/segments from 617902 to 616756 lines based on filter /tmp/kaldi.jxRb/recordings.
utils/fix_data_dir.sh: filtered data/train/wav.scp from 35606 to 35604 lines based on filter /tmp/kaldi.jxRb/recordings.
fix_data_dir.sh: kept 326122 utterances out of 617902
utils/fix_data_dir.sh: filtered data/train/wav.scp from 35604 to 35601 lines based on filter /tmp/kaldi.jxRb/recordings.
fix_data_dir.sh: old files are kept in data/train/.backup


cat: data/FSC/lexicon/lexicon.txt: No such file or directory
cat: /storage07/user_data/angfederico01/local/g2p-seq2seq_OLD/KIT_dict.norm: No such file or directory


CPU times: user 4.21 ms, sys: 949 µs, total: 5.16 ms
Wall time: 12.2 s


In [3]:
%%time
%%bash

. ./cmd.sh
. ./path.sh
set -e

FSC=data/FSC
ISIP=$(echo $FSC | sed 's/FSC/ISIP_TGL/g')
KIT=$(echo $FSC | sed 's/FSC/KIT_TGL/g')



# Audio data directory check
if [ ! -d $FSC ]; then
 echo "Error: run.sh requires a directory argument"
  exit 1;
fi

export LC_ALL=C;
# FSC dictionary file check
#if [ ! -f $dir/lexicon.txt ]; then
#  cat data/local/dict/finaldict.txt | sort --parallel=8 | uniq >$dir/lexicon.txt || exit 1;
#fi

fscdir=data/local/fsc
rm -fr $fscdir
mkdir -p $fscdir
cat $FSC/*/*/*-wav.list 2>/dev/null | sed '/^$/d' | sed '/01_xx01xxxx_14A/d' |\
    sed '/04_xx10xxxx_22B/d' | sed '/12_xx10xxxx_14B/d' |\
    sed 's/ /\\ /g' | sed 's/\([()]\)/\\\1/g' > $fscdir/wav.flist # Using All data

isipdir=data/local/isip
rm -fr $isipdir
mkdir -p $isipdir
ls $ISIP/*/*/*-wav.list | grep -v '[Ss]po' | xargs cat 2>/dev/null | sed '/^$/d' | sed 's/ /\\ /g' |\
    sort --parallel=8 | uniq > $isipdir/wav.flist

kitdir=data/local/kit
rm -fr $kitdir
mkdir -p $kitdir
cat $KIT/*/*/*-wav.list 2>/dev/null | sed '/^$/d' | sort --parallel=8 | uniq > $kitdir/wav.flist



# transcriptions
cat $FSC/*/*/*-trn.txt |\
    sed '/01_xx01xxxx_14A/d' | sed '/04_xx10xxxx_22B/d' | sed '/12_xx10xxxx_14B/d' |\
    awk '{if (($4 !~ /^\.\.$/) && ($4 !~ /^$/) && ($4 !~ /^<sp>$/)) \
        {printf "%s ",$1; for (i=4; i<=NF; i++) printf " %s",$i; printf "\n"}}' |\
    sed 's/AS/C/g; s/PB/O/g; s/PBAS/O/g' | sort --parallel=8 > $fscdir/text

ls $ISIP/*/*/*-trn.txt | grep -v '[Ss]po' | xargs cat 2>/dev/null | awk -F'\t' '{printf "%s  %s\n",$1,$4}' |\
    sort --parallel=8 > $isipdir/text

awk -F'\t' '{printf "%s  %s\n",$1,$4}' $KIT/*/*/*-trn.txt | sort --parallel=8 > $kitdir/text

CPU times: user 1.91 ms, sys: 2.5 ms, total: 4.41 ms
Wall time: 1.13 s


In [4]:
%%time
%%bash

. ./cmd.sh
. ./path.sh
set -e

FSC=data/FSC
ISIP=$(echo $FSC | sed 's/FSC/ISIP_TGL/g')
KIT=$(echo $FSC | sed 's/FSC/KIT_TGL/g')

fscdir=data/local/fsc
export LC_ALL=C;
# segments
cat $FSC/*/*/*-trn.txt |\
    sed '/01_xx01xxxx_14A/d' | sed '/04_xx10xxxx_22B/d' |\
    sed '/12_xx10xxxx_14B/d' | sed 's/AS/C/g; s/PB/O/g; s/PBAS/O/g' |\
    awk '{if (($4 !~ /^\.\.$/) && ($4 !~ /^$/) && ($4 !~ /^<sp>$/)) { \
	if ($3 > $2) {
	  segment=$1
	  split(segment,S,"[_]");
	  spkid=S[1]"_"S[2]"_"S[3];
	  print segment " " spkid " " $2 " " $3
	}
}}' | sort --parallel=8 > $fscdir/segments

isipdir=data/local/isip
ls $ISIP/*/*/*-trn.txt | grep -v '[Ss]po' | xargs cat 2>/dev/null |\
awk '{
          segment=$1
          #split(segment,S,"[.]");
          #spkid=S[1]"."S[2];
          print segment " " segment " " $2 " " $3
}' | sort --parallel=8 > $isipdir/segments

kitdir=data/local/kit
awk '{
          segment=$1
          split(segment,S,"[.]");
          spkid=S[1]"."S[2];
          print segment " " spkid " " $2 " " $3
}' <(cat $KIT/*/*/*-trn.txt) | sort --parallel=8 > $kitdir/segments



CPU times: user 0 ns, sys: 4.43 ms, total: 4.43 ms
Wall time: 1.05 s


In [5]:
%%time
%%bash

export LC_ALL=C;
fscdir=data/local/fsc
awk '{segment=$1; split(segment,S,"[_]"); spkid=S[1]"_"S[2]"_"S[3]; print $1 " " spkid}' $fscdir/segments |\
    sort --parallel=8 -k2 > $fscdir/utt2spk || exit 1;

isipdir=data/local/isip
awk '{segment=$1; split(segment,S,"[.]"); spkid=S[1]"."S[2]"."S[3]"."S[4]; print $1 " " spkid}' $isipdir/segments |\
    sort --parallel=8 -k2 > $isipdir/utt2spk || exit 1;

kitdir=data/local/kit
awk '{segment=$1; split(segment,S,"[.]"); spkid=S[1]"."S[2]; print $1 " " spkid}' $kitdir/segments |\
    sort --parallel=8 -k2 > $kitdir/utt2spk || exit 1;



CPU times: user 754 µs, sys: 3.15 ms, total: 3.91 ms
Wall time: 617 ms


In [6]:
%%time
%%bash

export LC_ALL=C;
fscdir=data/local/fsc
sed -e 's?.*/??' -e 's?.wav??' -e 's/PBAS/O/g; s/AS/C/g; s/PB/O/g;' $fscdir/wav.flist |\
    paste - $fscdir/wav.flist | sort --parallel=8 > $fscdir/wavflist.scp

isipdir=data/local/isip
sed -e 's?.*/??' -e 's?.wav??' $isipdir/wav.flist | paste - $isipdir/wav.flist |\
    sort --parallel=8 > $isipdir/wavflist.scp

kitdir=data/local/kit
sed -e 's?.*/\([^/]*\)/\([^/]*$\)?\1.\2?' -e 's?.wav??' $kitdir/wav.flist |\
    paste - $kitdir/wav.flist | sort --parallel=8 > $kitdir/wavflist.scp



CPU times: user 3.35 ms, sys: 946 µs, total: 4.29 ms
Wall time: 237 ms


In [ ]:
%%time
%%bash

. ./cmd.sh
. ./path.sh
set -e

dir=data/local/train
rm -fr $dir
mkdir -p $dir

export LC_ALL=C;
# FSC dictionary file check
if [ ! -f $dir/lexicon.txt ]; then
  cat data/local/dict/finaldict.txt | sort --parallel=8 | uniq >$dir/lexicon.txt || exit 1;
fi

fscdir=data/local/fsc
isipdir=data/local/isip
kitdir=data/local/kit

# recordings
cat $fscdir/wav.flist $isipdir/wav.flist $kitdir/wav.flist | sort --parallel=8 > $dir/wav.flist

# trans
cat $fscdir/text $isipdir/text $kitdir/text | sort --parallel=8 > $dir/text

# time start, time ends
cat $fscdir/segments $isipdir/segments $kitdir/segments | sort --parallel=8 > $dir/segments

cat $fscdir/utt2spk $isipdir/utt2spk $kitdir/utt2spk | sort --parallel=8 > $dir/utt2spk

cat $fscdir/wavflist.scp $isipdir/wavflist.scp $kitdir/wavflist.scp | sort --parallel=8 > $dir/wavflist.scp

In [ ]:
%%time
%%bash

dir=data/local/train

awk '{
 printf("%s cat ", $1);
 for (i=2;i<=NF;i++) {
    printf("%s ",$i)
 }
 printf("|\n")
}' < $dir/wavflist.scp | sort > $dir/wav.scp || exit 1;

sort -k 2 $dir/utt2spk | utils/utt2spk_to_spk2utt.pl > $dir/spk2utt || exit 1;

# Copy stuff into its final locations [this has been moved from the format_data script]
rm -fr data/train
mkdir -p data/train
for f in spk2utt utt2spk wav.scp text segments; do
  cp data/local/train/$f data/train/ || exit 1;
done

echo "FSC data preparation succeeded."

utils/fix_data_dir.sh data/train

In [ ]:
%%time
%%bash

. ./cmd.sh
. ./path.sh
set -e

local/fph_prepare_dict.sh

In [ ]:
%%time
%%bash

. ./cmd.sh
. ./path.sh
set -e # exit on error

rm -fr data/local/lang_nosilp
rm -fr data/lang_nosilp
utils/prepare_lang.sh --num-sil-states 4 data/local/dict_nosilp "<unk>" data/local/lang_nosilp data/lang_nosilp

In [ ]:
%%time
%%bash

. ./cmd.sh
. ./path.sh
set -e # exit on error

# Now make MFCC features.
# mfccdir should be some place with a largish disk where you
# want to store MFCC features.
mfccdir=mfcc

for x in train; do
  steps/make_mfcc.sh --nj 24 --cmd "$train_cmd" \
    data/$x exp/make_mfcc/$x $mfccdir
  steps/compute_cmvn_stats.sh data/$x exp/make_mfcc/$x $mfccdir
  utils/fix_data_dir.sh data/$x
done

echo "Finish creating MFCCs"

#SKIP

In [ ]:
%%time
%%bash

. ./cmd.sh
. ./path.sh
set -e # exit on error

#: << '#SKIP'

use_dev=false # Use the first 4k sentences from training data as dev set. (39 speakers.)

##### Training and Decoding steps start from here #####

# Use the first 4k sentences as dev set.  Note: when we trained the LM, we used
# the 1st 10k sentences as dev set, so the 1st 4k won't have been used in the
# LM training data.   However, they will be in the lexicon, plus speakers
# may overlap, so it's still not quite equivalent to a test set.
rm -fr data/train_nodev
if $use_dev ;then
    #dev_set=train_dev
    #utils/subset_data_dir.sh --first data/train 4000 data/$dev_set # 6hr 31min
    #n=$[`cat data/train/segments | wc -l` - 4000]
    #utils/subset_data_dir.sh --last data/train $n data/train_nodev
    cp -r data/train data/train_nodev
else
    cp -r data/train data/train_nodev
fi

In [ ]:
%%time
%%bash

. ./cmd.sh
. ./path.sh
set -e # exit on error

## Starting basic training on MFCC features
rm -fr exp/mono
steps/train_mono.sh --nj 24 --cmd "$train_cmd" \
  data/train_nodev data/lang_nosilp exp/mono

In [ ]:
%%time
%%bash

. ./cmd.sh
. ./path.sh
set -e # exit on error
rm -fr exp/mono_ali
rm -fr exp/tri1

steps/align_si.sh --nj 24 --cmd "$train_cmd" \
  data/train_nodev data/lang_nosilp exp/mono exp/mono_ali

steps/train_deltas.sh --cmd "$train_cmd" \
  3200 30000 data/train_nodev data/lang_nosilp exp/mono_ali exp/tri1

In [ ]:
%%time
%%bash

. ./cmd.sh
. ./path.sh
set -e # exit on error

rm -fr exp/tri1_ali
rm -fr exp/tri2

steps/align_si.sh --nj 24 --cmd "$train_cmd" \
  data/train_nodev data/lang_nosilp exp/tri1 exp/tri1_ali

steps/train_deltas.sh --cmd "$train_cmd" \
  4000 70000 data/train_nodev data/lang_nosilp exp/tri1_ali exp/tri2

In [ ]:
%%time
%%bash

. ./cmd.sh
. ./path.sh
set -e # exit on error

# From now, we start using all of the data (except some duplicates of common
# utterances, which don't really contribute much).
steps/align_si.sh --nj 24 --cmd "$train_cmd" \
  data/train_nodev data/lang_nosilp exp/tri2 exp/tri2_ali

# Do another iteration of LDA+MLLT training, on all the data.
steps/train_lda_mllt.sh --cmd "$train_cmd" \
  6000 140000 data/train_nodev data/lang_nosilp exp/tri2_ali exp/tri3

In [ ]:
%%time
%%bash

. ./cmd.sh
. ./path.sh
set -e # exit on error

# Now we compute the pronunciation and silence probabilities from training data,
# and re-create the lang directory.
steps/get_prons.sh --cmd "$train_cmd" data/train_nodev data/lang_nosilp exp/tri3
utils/dict_dir_add_pronprobs.sh --max-normalize true \
  data/local/dict_nosilp exp/tri3/pron_counts_nowb.txt exp/tri3/sil_counts_nowb.txt \
  exp/tri3/pron_bigram_counts_nowb.txt data/local/dict

In [ ]:
%%time
%%bash

. ./cmd.sh
. ./path.sh
set -e # exit on error

# Now we re-create the lang directory.
utils/prepare_lang.sh data/local/dict "<unk>" data/local/lang data/lang

In [ ]:
%%time
%%bash

. ./cmd.sh
. ./path.sh
set -e # exit on error

# Train tri4, which is LDA+MLLT+SAT, on all the (nodup) data.
steps/align_fmllr.sh --nj 24 --cmd "$train_cmd" \
  data/train_nodev data/lang exp/tri3 exp/tri3_ali

steps/train_sat.sh  --cmd "$train_cmd" \
  11500 200000 data/train_nodev data/lang exp/tri3_ali exp/tri4

### iVector common starts here ###

In [ ]:
%%time
%%bash

#set -euo pipefail

# This script is called from local/nnet3/run_tdnn.sh and local/chain/run_tdnn.sh
# (and may eventually be called by more scripts). It contains the common feature
# preparation and ivector-related parts of the script. See those scripts for 
# examples of usage.

stage=0
train_set=train_nodev
dev_set=
test_sets="eval1 eval2 eval3"
gmm=tri4

nnet3_affix=

. ./cmd.sh
. ./path.sh
. ./utils/parse_options.sh

if [ -e data/train_dev ] ;then
  dev_set=train_dev
fi

gmm_dir=exp/${gmm}
ali_dir=exp/${gmm}_ali_${train_set}_sp

for f in data/${train_set}/feats.scp ${gmm_dir}/final.mdl; do
  if [ ! -f $f ]; then
    echo "$0: expected file $f to exist"
    exit 1
  fi
done

if [ $stage -le 1 ]; then
  # Although the nnet will be trained by high resolution data, we still have to perturb
  # the normal data to get the alignment _sp stands for speed-perturbed
  echo "$0: preparing directory for low-resolution speed-perturbed data (for alignment)"
  utils/data/perturb_data_dir_speed_3way.sh data/${train_set} data/${train_set}_sp
  echo "$0: making MFCC featuresfor low-resolution speed-perturbed data"
  steps/make_mfcc.sh --cmd "$train_cmd" --nj 24 data/${train_set}_sp || exit 1;
  steps/compute_cmvn_stats.sh data/${train_set}_sp || exit 1;
  utils/fix_data_dir.sh data/${train_set}_sp
fi

In [ ]:
%%time
%%bash

#set -euo pipefail

stage=0
train_set=train_nodev
gmm=tri4

. ./cmd.sh
. ./path.sh
. ./utils/parse_options.sh

gmm_dir=exp/${gmm}
ali_dir=exp/${gmm}_ali_${train_set}_sp

if [ $stage -le 2 ]; then
  echo "$0: aligning with the perturbed low-resolution data"
  steps/align_fmllr.sh --nj 24 --cmd "$train_cmd" \
    data/${train_set}_sp data/lang $gmm_dir $ali_dir || exit 1;
fi

In [ ]:
%%time
%%bash

#set -euo pipefail

stage=0
train_set=train_nodev
dev_set=
test_sets="eval1 eval2 eval3"
gmm=tri4

. ./cmd.sh
. ./path.sh
. ./utils/parse_options.sh

if [ $stage -le 3 ]; then
  #Create high-resolution MFCC features (with 40 cepstra instead of 13).
  # this shows how you can split across multiple file-systems.
  echo "$0: creating high-resolution MFCC features"
  mfccdir=data/${train_set}_sp_hires/data

  for datadir in ${train_set}_sp; do
    utils/copy_data_dir.sh data/$datadir data/${datadir}_hires
  done

  # do volume-perturbation on the training data prior to extracting hires
  # features; this helps make trained nnets more invariant to test data volume
  utils/data/perturb_data_dir_volume.sh data/${train_set}_sp_hires || exit 1;

  # generate high-resolution MFCC feautres
  for datadir in ${train_set}_sp; do
    steps/make_mfcc.sh --nj 24 --mfcc-config conf/mfcc_hires.conf \
      --cmd "$train_cmd" data/${datadir}_hires || exit 1;
    steps/compute_cmvn_stats.sh data/${datadir}_hires || exit 1;
    utils/fix_data_dir.sh data/${datadir}_hires || exit 1;
  done
fi

In [ ]:
%%time
%%bash

#set -euo pipefail

stage=0
train_set=train_nodev
dev_set=
test_sets="eval1 eval2 eval3"
gmm=tri4

nnet3_affix=

. ./cmd.sh
. ./path.sh
. ./utils/parse_options.sh

if [ $stage -le 4 ]; then
  echo "$0: train the diagonal UBM."
  # There's no data, so use all
  mkdir -p exp/nnet3${nnet3_affix}/diag_ubm
  temp_data_root=exp/nnet3${nnet3_affix}/diag_ubm

  num_utts_total=$(wc -l <data/${train_set}_sp_hires/utt2spk)
  num_utts=$[$num_utts_total/1]
  utils/data/subset_data_dir.sh data/${train_set}_sp_hires \
    $num_utts ${temp_data_root}/${train_set}_sp_hires_subset

  echo "$0: computing a PCA transform from the hires data."
  steps/online/nnet2/get_pca_transform.sh --cmd "$train_cmd" \
    --splice-opts "--left-context=4 --right-context=4" \
    --max-utts 10000 --subsample 2 \
    ${temp_data_root}/${train_set}_sp_hires_subset \
    exp/nnet3${nnet3_affix}/pca_transform

  echo "$0: training the diagonal UBM."
  # Use 512 Gaussians in the UBM.
  steps/online/nnet2/train_diag_ubm.sh --cmd "$train_cmd" --nj 24 \
    --num-frames 500000 --num-threads 8 \
    ${temp_data_root}/${train_set}_sp_hires_subset 512 \
    exp/nnet3${nnet3_affix}/pca_transform exp/nnet3${nnet3_affix}/diag_ubm
fi

In [ ]:
%%time
%%bash

#set -euo pipefail

stage=0
train_set=train_nodev
dev_set=
test_sets="eval1 eval2 eval3"
gmm=tri4

nnet3_affix=

. ./cmd.sh
. ./path.sh
. ./utils/parse_options.sh

if [ $stage -le 5 ]; then
  # Train the iVector extractor.Use all of the speed-perturbed data since iVector extractors
  # can be sensitive to the amount of data. The script defaults to an iVector dimension of 100
  # even though $nj is just 2 (10 on high end set ups), each job uses multiple processes and threads.
  echo "$0: training the iVector extractor"
  steps/online/nnet2/train_ivector_extractor.sh --cmd "$train_cmd" --nj 2 \
    data/${train_set}_sp_hires exp/nnet3${nnet3_affix}/diag_ubm \
    exp/nnet3${nnet3_affix}/extractor || exit 1;
fi

In [ ]:
%%time
%%bash

#set -euo pipefail

stage=0
train_set=train_nodev
dev_set=
test_sets="eval1 eval2 eval3"
gmm=tri4

nnet3_affix=

. ./cmd.sh
. ./path.sh
. ./utils/parse_options.sh

if [ $stage -le 6 ]; then
  # We extract iVectors on the speed-perturbed training data after combining
  # short segments, which will be what we train the system on. With --utts-per-spk-max 2,
  # the scripts pairs the utterances into twos, and treats each of these pairs
  # as one speaker; this gives more diversity in iVectors.
  # Note that these are extracted 'online'.

  # Note, we don't encode the 'max2' in the name of the iVectordir even though
  # that's the data we extract the iVectors from, as it's still going to be 
  # valid for the non-'max2' data, the utterance list is the same.
  ivectordir=exp/nnet3${nnet3_affix}/ivectors_${train_set}_sp_hires

  # having a larger number of speakers is helpful for generalization, and to
  # handle per-utterance decoding well (iVector starts at zero).
  temp_data_root=${ivectordir}
  utils/data/modify_speaker_info.sh --utts-per-spk-max 2 \
    data/${train_set}_sp_hires ${temp_data_root}/${train_set}_sp_hires_max2

  steps/online/nnet2/extract_ivectors_online.sh --cmd "$train_cmd" --nj 24 \
    ${temp_data_root}/${train_set}_sp_hires_max2 \
    exp/nnet3${nnet3_affix}/extractor $ivectordir

  # Also extract iVectors for the test data, but in this case we don't need the speed
  # perturbation (sp).
  #for datadir in $dev_set $test_sets; do
  #  steps/online/nnet2/extract_ivectors_online.sh --cmd "$train_cmd" --nj 10 \
  #    data/${datadir}_hires exp/nnet3${nnet3_affix}/extractor \
  #    exp/nnet3${nnet3_affix}/ivectors_${datadir}_hires
  #done
fi

### Neural net training starts here

In [ ]:
%%time
%%bash

#set -euo pipefail

# First the options that are passed through to run_ivector_common.sh
# (some of which are also used in this script directly).
gmm=tri4
train_set=train_nodev
nnet3_affix=
# The rest are configs specific to this script. Most of the parameters
# are just hardcoded at this level, in the commands below.
affix=7n  # affix for the TDNN directory name
tree_affix=

# End configuration section.
echo "$0 $@"  # Print the command line for logging

. ./cmd.sh
. ./path.sh
. ./utils/parse_options.sh

if ! cuda-compiled; then
  cat <<EOF && exit 1
This script is intended to be used with GPUs but you have not compiled Kaldi with CUDA
If you want to use GPUs (and have them), go to src/, and configure and make on a machine
where "nvcc" is installed.
EOF
fi


gmm_dir=exp/$gmm
ali_dir=exp/${gmm}_ali_${train_set}_sp
tree_dir=exp/chain${nnet3_affix}/tree${tree_affix:+_$tree_affix}
lang=data/lang_chain
lat_dir=exp/chain${nnet3_affix}/${gmm}_${train_set}_sp_lats
dir=exp/chain${nnet3_affix}/tdnn${affix}
train_data_dir=data/${train_set}_sp_hires
lores_train_data_dir=data/${train_set}_sp
train_ivector_dir=exp/nnet3${nnet3_affix}/ivectors_${train_set}_sp_hires

for fa in $gmm_dir/final.mdl $train_data_dir/feats.scp $train_ivector_dir/ivector_online.scp \
  $lores_train_data_dir/feats.scp $ali_dir/ali.1.gz; do
  # unfortunately, this check won't work on jupyter's bash magic
  #[ ! -f $fa ] && echo "$0; expected file $fa to exist" && exit 1;
  ls -ltrh $fa
done

In [ ]:
%%time
%%bash

#set -euo pipefail

# First the options that are passed through to run_ivector_common.sh
# (some of which are also used in this script directly).
stage=0
gmm=tri4
train_set=train_nodev
nnet3_affix=
# The rest are configs specific to this script. Most of the parameters
# are just hardcoded at this level, in the commands below.
affix=7n  # affix for the TDNN directory name
tree_affix=

# End configuration section.
echo "$0 $@"  # Print the command line for logging

. ./cmd.sh
. ./path.sh
. ./utils/parse_options.sh

if ! cuda-compiled; then
  cat <<EOF && exit 1
This script is intended to be used with GPUs but you have not compiled Kaldi with CUDA
If you want to use GPUs (and have them), go to src/, and configure and make on a machine
where "nvcc" is installed.
EOF
fi


gmm_dir=exp/$gmm
ali_dir=exp/${gmm}_ali_${train_set}_sp
tree_dir=exp/chain${nnet3_affix}/tree${tree_affix:+_$tree_affix}
lang=data/lang_chain
lat_dir=exp/chain${nnet3_affix}/${gmm}_${train_set}_sp_lats
dir=exp/chain${nnet3_affix}/tdnn${affix}
train_data_dir=data/${train_set}_sp_hires
lores_train_data_dir=data/${train_set}_sp
train_ivector_dir=exp/nnet3${nnet3_affix}/ivectors_${train_set}_sp_hires

if [ $stage -le 9 ]; then
  # Get the alignments as lattices (gives the LF-MMI training more freedom).
  # use the same num-jobs as the alignments
  steps/align_fmllr_lats.sh --nj 24 --cmd "$train_cmd" ${lores_train_data_dir} \
    data/lang $gmm_dir $lat_dir
  rm $lat_dir/fsts.*.gz # save space
fi

In [ ]:
%%time
%%bash

#set -euo pipefail

# First the options that are passed through to run_ivector_common.sh
# (some of which are also used in this script directly).
stage=0

# End configuration section.
echo "$0 $@"  # Print the command line for logging

. ./cmd.sh
. ./path.sh
. ./utils/parse_options.sh

if ! cuda-compiled; then
  cat <<EOF && exit 1
This script is intended to be used with GPUs but you have not compiled Kaldi with CUDA
If you want to use GPUs (and have them), go to src/, and configure and make on a machine
where "nvcc" is installed.
EOF
fi

lang=data/lang_chain

if [ $stage -le 10 ]; then
  # Create a version of the lang/ directory that has one state per phone in the
  # topo file. [note, it really has two states.. the first one is only repeated
  # once, the second one has zero or more repeats.]
  rm -rf $lang
  cp -r data/lang $lang
  silphonelist=$(cat $lang/phones/silence.csl) || exit 1;
  nonsilphonelist=$(cat $lang/phones/nonsilence.csl) || exit 1;
  # Use our special topology... note that later on may have to tune this
  # topology.
  steps/nnet3/chain/gen_topo.py $nonsilphonelist $silphonelist >$lang/topo
fi


In [ ]:
%%time
%%bash

#set -euo pipefail

# First the options that are passed through to run_ivector_common.sh
# (some of which are also used in this script directly).
stage=0
decode_nj=10
train_set=train_nodev
dev_set=
test_sets="eval1 eval2 eval3"
gmm=tri4
nnet3_affix=

# The rest are configs specific to this script. Most of the parameters
# are just hardcoded at this level, in the commands below.
affix=7n  # affix for the TDNN directory name
tree_affix=
train_stage=-10
get_egs_stage=-10
decode_iter=

# training options
# training chunk-options
decode_iter=
num_epochs=10
initial_effective_lrate=0.001
final_effective_lrate=0.0001
leftmost_questions_truncate=-1
max_param_change=2.0
final_layer_normalize_target=0.5
num_jobs_initial=1
num_jobs_final=3
minibatch_size=128,64
#frames_per_eg=150,140,100
frames_per_eg=75,70,50
remove_egs=false
common_egs_dir=
xent_regularize=0.1

test_online_decoding=false  # if true, it will run the last decoding stage.

# End configuration section.
echo "$0 $@"  # Print the command line for logging

. ./cmd.sh
. ./path.sh
. ./utils/parse_options.sh

gmm_dir=exp/$gmm
ali_dir=exp/${gmm}_ali_${train_set}_sp
tree_dir=exp/chain${nnet3_affix}/tree${tree_affix:+_$tree_affix}
lang=data/lang_chain
lat_dir=exp/chain${nnet3_affix}/${gmm}_${train_set}_sp_lats
dir=exp/chain${nnet3_affix}/tdnn${affix}
train_data_dir=data/${train_set}_sp_hires
lores_train_data_dir=data/${train_set}_sp
train_ivector_dir=exp/nnet3${nnet3_affix}/ivectors_${train_set}_sp_hires

if [ $stage -le 11 ]; then
  # Build a tree using our new topology. This is the critically different
  # step compared with other recipes.
  steps/nnet3/chain/build_tree.sh --frame-subsampling-factor 3 \
      --leftmost-questions-truncate $leftmost_questions_truncate \
      --context-opts "--context-width=2 --central-position=1" \
      --cmd "$train_cmd" 7000 ${lores_train_data_dir} $lang $ali_dir $tree_dir
fi

In [ ]:
%%time
%%bash

#set -euo pipefail

# First the options that are passed through to run_ivector_common.sh
# (some of which are also used in this script directly).
stage=0
decode_nj=10
train_set=train_nodev
dev_set=
test_sets="eval1 eval2 eval3"
gmm=tri4
nnet3_affix=

# The rest are configs specific to this script. Most of the parameters
# are just hardcoded at this level, in the commands below.
affix=7n  # affix for the TDNN directory name
tree_affix=
train_stage=-10
get_egs_stage=-10
decode_iter=

# training options
# training chunk-options
decode_iter=
num_epochs=10
initial_effective_lrate=0.001
final_effective_lrate=0.0001
leftmost_questions_truncate=-1
max_param_change=2.0
final_layer_normalize_target=0.5
num_jobs_initial=1
num_jobs_final=3
minibatch_size=128,64
#frames_per_eg=150,140,100
frames_per_eg=75,70,50
remove_egs=false
common_egs_dir=
xent_regularize=0.1

test_online_decoding=false  # if true, it will run the last decoding stage.

# End configuration section.
echo "$0 $@"  # Print the command line for logging

. ./cmd.sh
. ./path.sh
. ./utils/parse_options.sh

gmm_dir=exp/$gmm
ali_dir=exp/${gmm}_ali_${train_set}_sp
tree_dir=exp/chain${nnet3_affix}/tree${tree_affix:+_$tree_affix}
lang=data/lang_chain
lat_dir=exp/chain${nnet3_affix}/${gmm}_${train_set}_sp_lats
dir=exp/chain${nnet3_affix}/tdnn${affix}
train_data_dir=data/${train_set}_sp_hires
lores_train_data_dir=data/${train_set}_sp
train_ivector_dir=exp/nnet3${nnet3_affix}/ivectors_${train_set}_sp_hires

if [ $stage -le 12 ]; then
  echo "$0: creating neural net configs using the xconfig parser";

  num_targets=$(tree-info $tree_dir/tree |grep num-pdfs|awk '{print $2}')
  learning_rate_factor=$(echo "print 0.5/$xent_regularize" | python2)
  opts="l2-regularize=0.002"
  linear_opts="orthonormal-constraint=1.0"
  output_opts="l2-regularize=0.0005 bottleneck-dim=256"

  mkdir -p $dir/configs

  cat <<EOF > $dir/configs/network.xconfig
  input dim=100 name=ivector
  input dim=40 name=input

  # please note that it is important to have input layer with the name=input
  # as the layer immediately preceding the fixed-affine-layer to enable
  # the use of short notation for the descriptor
  fixed-affine-layer name=lda input=Append(-1,0,1,ReplaceIndex(ivector, t, 0)) affine-transform-file=$dir/configs/lda.mat

  # the first splicing is moved before the lda layer, so no splicing here
  relu-batchnorm-layer name=tdnn1 $opts dim=1280
  linear-component name=tdnn2l dim=256 $linear_opts input=Append(-1,0)
  relu-batchnorm-layer name=tdnn2 $opts input=Append(0,1) dim=1280
  linear-component name=tdnn3l dim=256 $linear_opts
  relu-batchnorm-layer name=tdnn3 $opts dim=1280
  linear-component name=tdnn4l dim=256 $linear_opts input=Append(-1,0)
  relu-batchnorm-layer name=tdnn4 $opts input=Append(0,1) dim=1280
  linear-component name=tdnn5l dim=256 $linear_opts
  relu-batchnorm-layer name=tdnn5 $opts dim=1280 input=Append(tdnn5l, tdnn3l)
  linear-component name=tdnn6l dim=256 $linear_opts input=Append(-3,0)
  relu-batchnorm-layer name=tdnn6 $opts input=Append(0,3) dim=1280
  linear-component name=tdnn7l dim=256 $linear_opts input=Append(-3,0)
  relu-batchnorm-layer name=tdnn7 $opts input=Append(0,3,tdnn6l,tdnn4l,tdnn2l) dim=1280
  linear-component name=tdnn8l dim=256 $linear_opts input=Append(-3,0)
  relu-batchnorm-layer name=tdnn8 $opts input=Append(0,3) dim=1280
  linear-component name=tdnn9l dim=256 $linear_opts input=Append(-3,0)
  relu-batchnorm-layer name=tdnn9 $opts input=Append(0,3,tdnn8l,tdnn6l,tdnn4l) dim=1280
  linear-component name=tdnn10l dim=256 $linear_opts input=Append(-3,0)
  relu-batchnorm-layer name=tdnn10 $opts input=Append(0,3) dim=1280
  linear-component name=tdnn11l dim=256 $linear_opts input=Append(-3,0)
  relu-batchnorm-layer name=tdnn11 $opts input=Append(0,3,tdnn10l,tdnn8l,tdnn6l) dim=1280
  linear-component name=prefinal-l dim=256 $linear_opts

  relu-batchnorm-layer name=prefinal-chain input=prefinal-l $opts dim=1280
  output-layer name=output include-log-softmax=false dim=$num_targets $output_opts

  relu-batchnorm-layer name=prefinal-xent input=prefinal-l $opts dim=1280
  output-layer name=output-xent dim=$num_targets learning-rate-factor=$learning_rate_factor $output_opts
EOF
  steps/nnet3/xconfig_to_configs.py --xconfig-file $dir/configs/network.xconfig --config-dir $dir/configs/
fi

In [ ]:
%%time
%%bash

#set -euo pipefail

# First the options that are passed through to run_ivector_common.sh
# (some of which are also used in this script directly).
stage=0
decode_nj=10
train_set=train_nodev
dev_set=
test_sets="eval1 eval2 eval3"
gmm=tri4
nnet3_affix=

# The rest are configs specific to this script. Most of the parameters
# are just hardcoded at this level, in the commands below.
affix=7n  # affix for the TDNN directory name
tree_affix=
train_stage=3330
get_egs_stage=-10
decode_iter=

# training options
# training chunk-options
decode_iter=
num_epochs=10
initial_effective_lrate=0.001
final_effective_lrate=0.0001
leftmost_questions_truncate=-1
max_param_change=2.0
final_layer_normalize_target=0.5
num_jobs_initial=1
num_jobs_final=1
minibatch_size=128,64
#frames_per_eg=150,140,100
frames_per_eg=75,70,50
remove_egs=false
common_egs_dir=
xent_regularize=0.1

test_online_decoding=false  # if true, it will run the last decoding stage.

# End configuration section.
echo "$0 $@"  # Print the command line for logging

. ./cmd.sh
. ./path.sh
. ./utils/parse_options.sh

gmm_dir=exp/$gmm
ali_dir=exp/${gmm}_ali_${train_set}_sp
tree_dir=exp/chain${nnet3_affix}/tree${tree_affix:+_$tree_affix}
lang=data/lang_chain
lat_dir=exp/chain${nnet3_affix}/${gmm}_${train_set}_sp_lats
dir=exp/chain${nnet3_affix}/tdnn${affix}
train_data_dir=data/${train_set}_sp_hires
lores_train_data_dir=data/${train_set}_sp
train_ivector_dir=exp/nnet3${nnet3_affix}/ivectors_${train_set}_sp_hires

if [ $stage -le 13 ]; then
  steps/nnet3/chain/train.py --stage $train_stage \
    --cmd "$train_cmd" \
    --feat.online-ivector-dir $train_ivector_dir \
    --feat.cmvn-opts "--norm-means=false --norm-vars=false" \
    --chain.xent-regularize $xent_regularize \
    --chain.leaky-hmm-coefficient 0.1 \
    --chain.l2-regularize 0.0 \
    --chain.apply-deriv-weights false \
    --chain.lm-opts="--num-extra-lm-states=2000" \
    --egs.dir "$common_egs_dir" \
    --egs.stage $get_egs_stage \
    --egs.opts "--frames-overlap-per-eg 0" \
    --egs.chunk-width $frames_per_eg \
    --trainer.num-chunk-per-minibatch 128 \
    --trainer.frames-per-iter 1500000 \
    --trainer.num-epochs $num_epochs \
    --trainer.optimization.num-jobs-initial $num_jobs_initial \
    --trainer.optimization.num-jobs-final $num_jobs_final \
    --trainer.optimization.initial-effective-lrate 0.001 \
    --trainer.optimization.final-effective-lrate 0.0001 \
    --trainer.max-param-change 2.0 \
    --use-gpu=wait \
    --cleanup.remove-egs $remove_egs \
    --feat-dir $train_data_dir \
    --tree-dir $tree_dir \
    --lat-dir $lat_dir \
    --dir $dir  || exit 1;

fi


In [ ]:
%%time
%%bash

lmDir=data/exp/lm
srcdict=data/exp/dict_bigtext/lexicon.txt
dir=data/exp/dict_bigtext

export LC_ALL=C

cat data/exp/dict_bigtext/ALL_dict | sed '/<sp>/d; /<unk>/d; /^[ ]*$/d' | sort --parallel=8 | uniq > $srcdict


cat $srcdict > $dir/lexicon1.txt || exit 1;

#cat $dir/lexicon1.txt | awk '{ for(n=2;n<=NF;n++){ phones[$n] = 1; }} END{for (p in phones) print p;}' | \
#  grep -v SIL > $dir/nonsilence_phones.txt  || exit 1;
cp data/local/dict/nonsilence_phones.txt $dir/nonsilence_phones.txt

#( echo sil; echo spn; echo nsn; echo lau ) > $dir/silence_phones.txt
#( echo SIL ; echo SPN ; ) > $dir/silence_phones.txt
cp data/local/dict/silence_phones.txt $dir/silence_phones.txt

#echo SIL > $dir/optional_silence.txt
cp data/local/dict/optional_silence.txt $dir/optional_silence.txt

# No "extra questions" in the input to this setup, as we don't
# have stress or tone.
#echo -n >$dir/extra_questions.txt
cp data/local/dict/extra_questions.txt $dir/extra_questions.txt

# Add to the lexicon the silences, noises etc.
( echo '<sp> SIL' ; echo '<unk> SPN'; ) | cat - $dir/lexicon1.txt  > $dir/lexicon2.txt || exit 1;

cp $dir/lexicon2.txt $dir/lexicon.txt

In [ ]:
%%time
%%bash

. ./cmd.sh
. ./path.sh
set -e # exit on error

rm -fr data/exp/lang_bigtext
utils/prepare_lang.sh --num-sil-states 4 data/exp/dict_bigtext "<unk>" data/exp/lang_bigtext/tmp data/exp/lang_bigtext

In [ ]:
%%time
%%bash

. ./cmd.sh
. ./path.sh
set -e # exit on error

dir=data/exp/lm
text=data/TEXT/EVERYTHING_UNIQ
lexicon=data/exp/dict_bigtext/lexicon.txt
mkdir -p $dir
export LC_ALL=C 

cat $text | gzip -c > $dir/train.all.gz
#cut -d' ' -f2- $text | tail -n +$heldout_sent | gzip -c > $dir/train.gz
#cut -d' ' -f2- $text | head -n $heldout_sent > $dir/heldout

cut -d' ' -f1 $lexicon > $dir/wordlist

ngram-count -text $dir/train.all.gz -order 4 -limit-vocab -vocab $dir/wordlist \
  -unk -map-unk "<unk>" -kndiscount -interpolate -lm $dir/fph.o4g.kn.gz
#echo "PPL for CSJ LM:"
#ngram -unk -lm $dir/csj.o3g.kn.gz -ppl $dir/heldout
#ngram -unk -lm $dir/csj.o3g.kn.gz -ppl $dir/heldout -debug 2 >& $dir/ppl2

In [ ]:
%%time
%%bash

. ./cmd.sh
. ./path.sh
set -e # exit on error

# We don't really need all these options for SRILM, since the LM training script
# does some of the same processing (e.g. -subset -tolower)
srilm_opts="-subset -prune-lowprobs -unk -tolower -order 4"
LM=data/exp/lm/fph.o4g.kn.gz
rm -fr data/exp/lang_nosilp_fph_4g
utils/format_lm_sri.sh --srilm-opts "$srilm_opts" \
  data/exp/lang_bigtext $LM data/exp/dict_bigtext/lexicon.txt data/exp/lang_nosilp_fph_4g

In [ ]:
%%time
%%bash

#set -euo pipefail

# First the options that are passed through to run_ivector_common.sh
# (some of which are also used in this script directly).
stage=0
decode_nj=10
train_set=train_nodev
dev_set=
test_sets="eval1 eval2 eval3"
gmm=tri4
nnet3_affix=

# The rest are configs specific to this script. Most of the parameters
# are just hardcoded at this level, in the commands below.
affix=7n  # affix for the TDNN directory name
tree_affix=
train_stage=-10
get_egs_stage=-10
decode_iter=

# training options
# training chunk-options
decode_iter=
num_epochs=10
initial_effective_lrate=0.001
final_effective_lrate=0.0001
leftmost_questions_truncate=-1
max_param_change=2.0
final_layer_normalize_target=0.5
num_jobs_initial=1
num_jobs_final=3
minibatch_size=128,64
#frames_per_eg=150,140,100
frames_per_eg=75,70,50
remove_egs=false
common_egs_dir=
xent_regularize=0.1

test_online_decoding=false  # if true, it will run the last decoding stage.

# End configuration section.
echo "$0 $@"  # Print the command line for logging

. ./cmd.sh
. ./path.sh
. ./utils/parse_options.sh

gmm_dir=exp/$gmm
ali_dir=exp/${gmm}_ali_${train_set}_sp
tree_dir=exp/chain${nnet3_affix}/tree${tree_affix:+_$tree_affix}
lang=data/lang_chain
lat_dir=exp/chain${nnet3_affix}/${gmm}_${train_set}_sp_lats
dir=exp/chain${nnet3_affix}/tdnn${affix}
train_data_dir=data/${train_set}_sp_hires
lores_train_data_dir=data/${train_set}_sp
train_ivector_dir=exp/nnet3${nnet3_affix}/ivectors_${train_set}_sp_hires

if [ $stage -le 14 ]; then
  # Note: it might appear that this $lang directory is mismatched, and it is as
  # far as the 'topo' is concerned, but this script doesn't read the 'topo' from
  # the lang directory.
  utils/mkgraph.sh --self-loop-scale 1.0 data/exp/lang_nosilp_fph_4g $dir $dir/graph_fph_4g
fi
